In [2]:
%load_ext sparksql_magic

In [3]:
#setup libs
import os
import findspark
from pyspark.sql import *
from pyspark import SparkConf, SparkContext
from py4j.java_gateway import java_import
findspark.init()

In [4]:
# AWS credentials - Use environment variables or AWS profile instead of hardcoding
# Credentials will be read from ~/.aws/credentials or environment variables
import os
os.environ['AWS_PROFILE'] = 'tarun_student'  # Use your AWS profile
aws_region = 'us-east-1'

In [ ]:
# AWS Profile is set in the cell above via os.environ
# Shell export commands don't persist in Jupyter notebooks

In [5]:
database_name = "prod"
table_name="orders"

In [6]:
from pyspark.sql import SparkSession
import os

# IMPORTANT: Stop any existing Spark session first
try:
    spark.stop()
    print("🛑 Stopped existing Spark session")
except:
    pass

# Set up JAR paths (using local JARs - same as working spark-iceberg-polaris-final.sh)
jars_dir = os.path.expanduser("~/iceberg/jars")
jars = [
    f"{jars_dir}/bundle-2.31.63.jar",
    f"{jars_dir}/url-connection-client-2.31.63.jar",
    f"{jars_dir}/iceberg-aws-1.10.0.jar",
    f"{jars_dir}/iceberg-spark-runtime-4.0_2.13-1.10.0.jar"
]

# Build driver classpath string
driver_classpath = ":".join(jars)

spark = (SparkSession.builder
  .appName("Iceberg-Polaris")
  # Use local JARs instead of Maven packages
  .config("spark.jars", ",".join(jars))
  .config("spark.driver.extraClassPath", driver_classpath)
  .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

  # Polaris catalog configuration (matching working spark-iceberg-polaris-final.sh)
  .config("spark.sql.defaultCatalog","polaris")
  .config("spark.sql.catalog.polaris","org.apache.iceberg.spark.SparkCatalog")
  .config("spark.sql.catalog.polaris.type","rest")
  .config("spark.sql.catalog.polaris.io-impl","org.apache.iceberg.aws.s3.S3FileIO")
  .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation","true")
  .config("spark.sql.catalog.polaris.uri","https://<>-ambaripolaris.snowflakecomputing.com/polaris/api/catalog")
  .config("spark.sql.catalog.polaris.credential","KoxyLE5rHsO5BenWbwb8=")
  .config("spark.sql.catalog.polaris.warehouse","ambari_prod")
  .config("spark.sql.catalog.polaris.scope","PRINCIPAL_ROLE:ALL")
  .config("spark.sql.catalog.polaris.s3.region","us-east-2")
  
  # Disable vectorization (for compatibility)
  .config("spark.sql.iceberg.vectorization.enabled","false")
  .getOrCreate())

print("✅ Spark session created successfully with Polaris catalog")


25/10/18 21:33:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark session created successfully with Polaris catalog


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60603)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/socketserver.py", line 766, in __init__
    self.handle()
    ~~~~~~~~~~~^^
  File "/Lib

In [7]:
from pyspark.sql import Row
import time

ut = time.time()

product = [
    {'id': '00001', 'name': 'Heater','price': 99900.76 },
    {'id': '00002', 'name': 'Thermostat','price': 881500.00},
    {'id': '00003', 'name': 'Television','price': 1400.89},
    {'id': '00004', 'name': 'Blender','price': 300.99},
    {'id': '00005', 'name': 'USB charger','price': 400.00}
]

df_products = spark.createDataFrame(Row(**x) for x in product)

In [8]:
df_products.createOrReplaceTempView("tmp_product")

In [9]:
# Query the temporary view
spark.sql("SELECT * FROM tmp_product").show()

+-----+-----------+--------+
|   id|       name|   price|
+-----+-----------+--------+
|00001|     Heater|99900.76|
|00002| Thermostat|881500.0|
|00003| Television| 1400.89|
|00004|    Blender|  300.99|
|00005|USB charger|   400.0|
+-----+-----------+--------+



In [10]:
spark.sql(f"""
CREATE NAMESPACE IF NOT EXISTS polaris.{database_name}""")

25/10/18 21:34:56 WARN AuthManagers: Inferring rest.auth.type=oauth2 since property credential was provided. Please explicitly set rest.auth.type to avoid this warning.


Py4JJavaError: An error occurred while calling o56.sql.
: org.apache.iceberg.exceptions.RESTException: Failed to create request URI from base https://<>-ambaripolaris.snowflakecomputing.com/polaris/api/catalog
	at org.apache.iceberg.rest.HTTPClient$Builder.uri(HTTPClient.java:488)
	at org.apache.iceberg.rest.RESTCatalog.lambda$new$0(RESTCatalog.java:60)
	at org.apache.iceberg.rest.RESTSessionCatalog.initialize(RESTSessionCatalog.java:203)
	at org.apache.iceberg.rest.RESTCatalog.initialize(RESTCatalog.java:82)
	at org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:280)
	at org.apache.iceberg.CatalogUtil.buildIcebergCatalog(CatalogUtil.java:337)
	at org.apache.iceberg.spark.SparkCatalog.buildIcebergCatalog(SparkCatalog.java:155)
	at org.apache.iceberg.spark.SparkCatalog.initialize(SparkCatalog.java:753)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:65)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:56)

25/10/19 03:28:17 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1000633 ms exceeds timeout 120000 ms
25/10/19 03:28:17 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/19 03:43:50 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS polaris.{database_name}.{table_name}")

In [ ]:
sql_stmnt = f"""
CREATE TABLE IF NOT EXISTS polaris.{database_name}.{table_name} (
    id string,
    name string,
    price decimal(10,2)
)
USING iceberg
TBLPROPERTIES (
'table_type'='ICEBERG', 
'format-version'='2'
)
LOCATION 's3://ambaricloudsatya/{database_name}/{table_name}/'
"""
print(f"Executing SparkSQL:\n`{sql_stmnt}`")
spark.sql(sql_stmnt).show()

In [ ]:
!aws s3 ls 's3://ambaricloudsatya/{database_name}/{table_name}' --recursive

In [ ]:
spark.sql(f"DESC EXTENDED polaris.{database_name}.{table_name}").show(truncate=False)

In [ ]:
spark.sql(f"""
INSERT INTO polaris.{database_name}.{table_name} 
SELECT * FROM tmp_product
""").show()

In [ ]:
spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name}").show()

In [ ]:
spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name}.history").show()

In [ ]:
spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name}.snapshots").show()

In [ ]:
spark.sql(f"""
SELECT h.made_current_at, s.operation, 
h.snapshot_id, h.is_current_ancestor, 
s.summary["spark.app.id"] 
FROM polaris.{database_name}.{table_name}.history h 
JOIN polaris.{database_name}.{table_name}.snapshots s  
ON h.snapshot_id = s.snapshot_id 
ORDER BY made_current_at
""").show()

In [ ]:
spark.sql(f'DELETE FROM polaris.{database_name}.{table_name} WHERE name = "Blender"').show()

In [ ]:
spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name}").show()

In [ ]:
ut = time.time()

product_updates = [
    {'id': '00001', 'name': 'Heater', 'price': 400}, # Update
    {'id': '00006', 'name': 'Chair', 'price': 500} # Insert
]
df_product_updates = spark.createDataFrame(Row(**x) for x in product_updates)

In [ ]:
df_product_updates.createOrReplaceTempView(f"tmp_prodct_updates")

In [ ]:
spark.sql("SELECT * FROM tmp_prodct_updates").show()

In [ ]:
query = f"""
MERGE INTO polaris.{database_name}.{table_name} AS t
USING (SELECT * FROM tmp_prodct_updates) AS u
ON t.id = u.id
WHEN MATCHED THEN UPDATE SET t.price = u.price
WHEN NOT MATCHED THEN INSERT *
"""
spark.sql(query)

In [ ]:
spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name}").show()

In [ ]:
spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name}.snapshots ORDER BY committed_at DESC").show()

In [ ]:
#spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name} TIMESTAMP AS OF '2025-10-19 10:22:03.382000'").show()

In [ ]:
# Time travel query example - uncomment and update timestamp as needed
# spark.sql(f"SELECT * FROM polaris.{database_name}.{table_name} TIMESTAMP AS OF '2025-10-19 10:22:03.382000'").show()